### Sources

http://billchambers.me/tutorials/2015/01/14/python-nlp-cheatsheet-nltk-scikit-learn.html

http://glowingpython.blogspot.de/2013/07/combining-scikit-learn-and-ntlk.html

http://www.cs.duke.edu/courses/spring14/compsci290/assignments/lab02.html

https://stackoverflow.com/questions/10526579/use-scikit-learn-to-classify-into-multiple-categories

https://github.com/anuraagvak/IRE-PersonalityRecognition-Final/blob/master/ire_report.pdf

https://github.com/Charudatt89/Personality_Recognition/blob/master/22-9-PersonalityRecognition/Report/Report.pdf

http://stackoverflow.com/questions/2148543/how-to-write-a-confusion-matrix-in-python

Lexical and syntactic features: http://www.aicbt.com/authorship-attribution/

In [22]:
import pandas as pd
import numpy as np
import re, reprlib, sys
import matplotlib.pyplot as plt
from itertools import *
import random as ran
from logging import *
from pprint import *
from time import *
from rpy2 import *

ran.seed(5125)

%load_ext cython
%load_ext cythonmagic
%matplotlib inline

from pandas_confusion import *
from scipy.cluster.vq import *

import nltk as n
import nltk, nltk.classify.util, nltk.metrics, nltk.tokenize, nltk.stem
from nltk.corpus import *
from nltk.stem import *
from nltk.classify import *
from nltk.collocations import *
from nltk.metrics import BigramAssocMeasures as BAM
from nltk.probability import *
from nltk.classify.scikitlearn import *
from nltk.tag.sequential import *
from nltk.tag import *
from nltk.tag.util import *


import sklearn as sk
from sklearn import *
from sklearn.feature_extraction.text import *
from sklearn.svm import *
from sklearn.pipeline import *
from sklearn.multiclass import *
from sklearn.naive_bayes import *
from sklearn.neighbors import *
from sklearn.feature_selection import *
from sklearn.ensemble import *
from sklearn.grid_search import *
from sklearn.linear_model import *
# n.download()

The cython extension is already loaded. To reload it, use:
  %reload_ext cython
The cythonmagic extension is already loaded. To reload it, use:
  %reload_ext cythonmagic


### Read data and show them

In [2]:
data = pd.read_csv("../raw_data/data.csv", parse_dates=True, infer_datetime_format=True, 
            sep = None, encoding = "latin-1", engine = "python")
data.head(n=2)
# data.describe()

,#AUTHID,STATUS,sEXT,sNEU,sAGR,sCON,sOPN,cEXT,cNEU,cAGR,cCON,cOPN,DATE,NETWORKSIZE,BETWEENNESS,NBETWEENNESS,DENSITY,BROKERAGE,NBROKERAGE,TRANSITIVITY
0,b7b7764cfa1c523e4e93ab2a79a946c4,likes the sound of thunder.,2.65,3,3.15,3.25,4.4,n,y,n,n,y,06/19/09 03:21 PM,180,14861.6,93.29,0.03,15661,0.49,0.1
1,b7b7764cfa1c523e4e93ab2a79a946c4,is so sleepy it's not even funny that's she ca...,2.65,3,3.15,3.25,4.4,n,y,n,n,y,07/02/09 08:41 AM,180,14861.6,93.29,0.03,15661,0.49,0.1


In [3]:
d = {'n': "no", 'y': "yes"}

# http://stackoverflow.com/a/17702781
data = data.replace(d)

neu = data[["#AUTHID","STATUS","cNEU"]]
ext = data[["#AUTHID","STATUS","cEXT"]]
agr = data[["#AUTHID","STATUS","cAGR"]]
con = data[["#AUTHID","STATUS","cCON"]]
opn = data[["#AUTHID","STATUS","cOPN"]]

### Split data into train + test


In [4]:
train_neu, test_neu = sk.cross_validation.train_test_split(neu, train_size = 0.66)
train_ext, test_ext = sk.cross_validation.train_test_split(ext, train_size = 0.66)
train_agr, test_agr = sk.cross_validation.train_test_split(agr, train_size = 0.66)
train_con, test_con = sk.cross_validation.train_test_split(con, train_size = 0.66)
train_opn, test_opn = sk.cross_validation.train_test_split(opn, train_size = 0.66)
# print(len(train_neu))

### Creates diff. pipelines here!

with stop_words : ‘english’ in CountVecto or T.Vecto accuracy ussualy lower

In [27]:
parametersSVC = {
    'tfidf__max_df': (0.5, 0.75, 1.0),
    #'tfidf__max_features': (None, 5000, 10000, 50000),
    'tfidf__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2', None),
    'tfidf__smooth_idf': (True, False),
    #'clf__loss': ("hinge", "squared_hinge"),
    #'clf__penalty': ('l1', 'l2'),
    'clf__C': (0.5, 0.75, 1.0),
    'clf__dual': (True, False),
    'tfidf__stop_words' : ("english", None),
    #'tfidf__lowercase' : (True, False),
    'clf__multi_class': ('ovr', 'crammer_singer')
}

In [18]:
target_names = ['yes', 'no']

# 3 Support Vector Machines
classifierLinearSVC = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC())]) # OneVsRestClassifier already implemented, docs: http://scikit-learn.org/stable/modules/svm.html#multi-class-classification

classifierNuSVC = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,2))),
    ('clf', NuSVC(cache_size=500))])

classifierSVC = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,2))),
    ('clf', SVC(cache_size=500))]) # "one-against-one", docs: http://scikit-learn.org/stable/modules/svm.html#multi-class-classification

# 2 Naive Bayes
classifierNB = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,2))),
    ('nb', MultinomialNB())])

classifierBNB = Pipeline([    
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,2))),
    ('bnb', BernoulliNB())])


# Nearest Neighboars
classifier_kNN = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,2))),
    ('kNN', KNeighborsClassifier())]) # http://scikit-learn.org/stable/modules/neighbors.html#neighbors

# 2 Ensamble methods
classifierRF = Pipeline([    
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,2))),
    ('rfc', RandomForestClassifier(n_jobs=-1))])

classifierADC = Pipeline([    
    ('vectorizer_tfidf', TfidfVectorizer(ngram_range = (1,2))),
    ('adc', AdaBoostClassifier())])


### cNEU

In [28]:
# Display progress logs on stdout
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(levelname)s %(message)s')

grid_search = GridSearchCV(classifierLinearSVC, parametersSVC, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in classifierLinearSVC.steps])
print("parameters:")
pprint(parametersSVC)
t0 = time()
grid_search.fit(neu["STATUS"], neu["cNEU"])
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parametersSVC.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
pipeline: ['tfidf', 'clf']
parameters:
{'clf__C': (0.5, 0.75, 1.0),
 'clf__dual': (True, False),
 'clf__multi_class': ('ovr', 'crammer_singer'),
 'tfidf__max_df': (0.5, 0.75, 1.0),
 'tfidf__ngram_range': ((1, 1), (1, 2)),
 'tfidf__norm': ('l1', 'l2', None),
 'tfidf__smooth_idf': (True, False),
 'tfidf__stop_words': ('english', None),
 'tfidf__use_idf': (True, False)}
Fitting 3 folds for each of 1728 candidates, totalling 5184 fits

[Parallel(n_jobs=-1)]: Done  49 tasks       | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 199 tasks       | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 449 tasks       | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 799 tasks       | elapsed: 25.2min
[Parallel(n_jobs=-1)]: Done 1249 tasks       | elapsed: 46.1min
[Parallel(n_jobs=-1)]: Done 1799 tasks       | elapsed: 79.8min
[Parallel(n_jobs=-1)]: Done 2449 tasks       | elapsed: 110.8min
[Parallel(n_jobs=-1)]: Done 3199 tasks       | elapsed: 154.8min
[Parallel(n_jobs=-1)]: Done 4049 tasks       | elapsed: 195.7min
[Parallel(n_jobs=-1)]: Done 4999 tasks       | elapsed: 255.6min



done in 15906.388s

Best score: 0.626
Best parameters set:
	clf__C: 0.5
	clf__dual: True
	clf__multi_class: 'crammer_singer'
	tfidf__max_df: 0.5
	tfidf__ngram_range: (1, 2)
	tfidf__norm: 'l1'
	tfidf__smooth_idf: True
	tfidf__stop_words: None
	tfidf__use_idf: True


[Parallel(n_jobs=-1)]: Done 5184 out of 5184 | elapsed: 265.0min finished


In [11]:
clas_fit_SVC = classifierSVC.fit(train_neu["STATUS"], train_neu["cNEU"])
predicted_SVC = classifierSVC.predict(test_neu["cNEU"])
scores_SVC = cross_validation.cross_val_score(classifierSVC, neu["STATUS"], neu["cNEU"], cv = 5)
print("Accuracy rbf kernel SVC: %0.3f (+/- %0.2f)" % (scores_SVC.mean(), scores_SVC.std() * 2))

clas_fit_LSVC = classifierLinearSVC.fit(train_neu["STATUS"], train_neu["cNEU"])
predicted_LSVC = classifierLinearSVC.predict(test_neu["cNEU"])
scores_LSVC = cross_validation.cross_val_score(classifierLinearSVC, neu["STATUS"], neu["cNEU"], cv = 5)
print("Accuracy LinearSVC: %0.3f (+/- %0.2f)" % (scores_LSVC.mean(), scores_LSVC.std() * 2))

clas_fit_NuSVC = classifierNuSVC.fit(train_neu["STATUS"], train_neu["cNEU"])
predicted_NuSVC = classifierNuSVC.predict(test_neu["cNEU"])
scores_NuSVC = cross_validation.cross_val_score(classifierNuSVC, neu["STATUS"], neu["cNEU"], cv = 5)
print("Accuracy NuSVC: %0.3f (+/- %0.2f)" % (scores_NuSVC.mean(), scores_NuSVC.std() * 2))

print("")

clas_fit_BNB = classifierBNB.fit(train_neu["STATUS"], train_neu["cNEU"])
predicted_BNB = classifierBNB.predict(test_neu["cNEU"])
scores_BNB = cross_validation.cross_val_score(classifierBNB, neu["STATUS"], neu["cNEU"], cv = 5)
print("Accuracy of bernuli NB: %0.3f (+/- %0.2f)" % (scores_BNB.mean(), scores_BNB.std() * 2))

clas_fit_NB = classifierNB.fit(train_neu["STATUS"], train_neu["cNEU"])
predicted_NB = classifierNB.predict(test_neu["cNEU"])
scores_NB = cross_validation.cross_val_score(classifierNB, neu["STATUS"], neu["cNEU"], cv = 5)
print("Accuracy NB: %0.3f (+/- %0.2f)" % (scores_NB.mean(), scores_NB.std() * 2))

print("")

clas_fit_kNN = classifier_kNN.fit(train_neu["STATUS"], train_neu["cNEU"])
predicted_kNN = classifier_kNN.predict(test_neu["cNEU"])
scores_kNN = cross_validation.cross_val_score(classifier_kNN, neu["STATUS"], neu["cNEU"], cv = 5)
print("Accuracy of kNN: %0.3f (+/- %0.2f)" % (scores_kNN.mean(), scores_kNN.std() * 2))

print("")

clas_fit_ADC = classifierADC.fit(train_neu["STATUS"], train_neu["cNEU"])
predicted_ADC = classifierADC.predict(test_neu["cNEU"])
scores_ADC = cross_validation.cross_val_score(classifierADC, neu["STATUS"], neu["cNEU"], cv = 5)
print("Accuracy of AdaBoost: %0.3f (+/- %0.2f)" % (scores_ADC.mean(), scores_ADC.std() * 2))

clas_fit_RF = classifierRF.fit(train_neu["STATUS"], train_neu["cNEU"])
predicted_RF = classifierRF.predict(test_neu["cNEU"])
scores_RF = cross_validation.cross_val_score(classifierRF, neu["STATUS"], neu["cNEU"], cv = 5)
print("Accuracy of random Forest: %0.3f (+/- %0.2f)" % (scores_RF.mean(), scores_RF.std() * 2))

Accuracy rbf kernel SVC: 0.625 (+/- 0.00)
Accuracy LinearSVC: 0.571 (+/- 0.02)
Accuracy NuSVC: 0.472 (+/- 0.08)

Accuracy of bernuli NB: 0.625 (+/- 0.00)
Accuracy NB: 0.624 (+/- 0.00)

Accuracy of kNN: 0.421 (+/- 0.05)

Accuracy of AdaBoost: 0.607 (+/- 0.02)
Accuracy of random Forest: 0.590 (+/- 0.03)


#### VotingClassifier cNUE

http://scikit-learn.org/stable/modules/ensemble.html#votingclassifier

In [12]:
eclf = VotingClassifier(estimators=[
        ("svc", clas_fit_SVC),
        ("linear_svc", clas_fit_LSVC), 
        ("nu_svc", clas_fit_NuSVC),
        ("bnb", clas_fit_BNB),
        ("nb", clas_fit_NB),
        ("kNN", clas_fit_kNN), 
        ("rf", clas_fit_RF), 
        ("adc", clas_fit_ADC)], voting='hard')


for clf, label in zip([clas_fit_SVC, clas_fit_LSVC, clas_fit_NuSVC, clas_fit_BNB, clas_fit_NB, clas_fit_kNN, clas_fit_RF, clas_fit_ADC, eclf], ["SVC", "LSVC","NuSVC","BNB","NB", 'kNN', 'Random Forest', 'ADC', 'Ensemble']):
    scores = cross_validation.cross_val_score(clf, neu["STATUS"], neu["cNEU"], cv=5, scoring='accuracy')
    print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.625 (+/- 0.00) [SVC]
Accuracy: 0.571 (+/- 0.01) [LSVC]
Accuracy: 0.472 (+/- 0.04) [NuSVC]
Accuracy: 0.625 (+/- 0.00) [BNB]
Accuracy: 0.624 (+/- 0.00) [NB]
Accuracy: 0.421 (+/- 0.03) [kNN]
Accuracy: 0.593 (+/- 0.02) [Random Forest]
Accuracy: 0.607 (+/- 0.01) [ADC]
Accuracy: 0.623 (+/- 0.00) [Ensemble]


### cAGR


In [13]:
clas_fit_SVC = classifierSVC.fit(train_agr["STATUS"], train_agr["cAGR"])
predicted_SVC = classifierSVC.predict(test_agr["cAGR"])
scores_SVC = cross_validation.cross_val_score(classifierSVC, agr["STATUS"], agr["cAGR"], cv = 5)
print("Accuracy rbf kernel SVC: %0.3f (+/- %0.2f)" % (scores_SVC.mean(), scores_SVC.std() * 2))

clas_fit_LSVC = classifierLinearSVC.fit(train_agr["STATUS"], train_agr["cAGR"])
predicted_LSVC = classifierLinearSVC.predict(test_agr["cAGR"])
scores_LSVC = cross_validation.cross_val_score(classifierLinearSVC, agr["STATUS"], agr["cAGR"], cv = 5)
print("Accuracy LinearSVC: %0.3f (+/- %0.2f)" % (scores_LSVC.mean(), scores_LSVC.std() * 2))

clas_fit_NuSVC = classifierNuSVC.fit(train_agr["STATUS"], train_agr["cAGR"])
predicted_NuSVC = classifierNuSVC.predict(test_agr["cAGR"])
scores_NuSVC = cross_validation.cross_val_score(classifierNuSVC, agr["STATUS"], agr["cAGR"], cv = 5)
print("Accuracy NuSVC: %0.3f (+/- %0.2f)" % (scores_NuSVC.mean(), scores_NuSVC.std() * 2))

print("")

clas_fit_BNB = classifierBNB.fit(train_agr["STATUS"], train_agr["cAGR"])
predicted_BNB = classifierBNB.predict(test_agr["cAGR"])
scores_BNB = cross_validation.cross_val_score(classifierBNB, agr["STATUS"], agr["cAGR"], cv = 5)
print("Accuracy of bernuli NB: %0.3f (+/- %0.2f)" % (scores_BNB.mean(), scores_BNB.std() * 2))

clas_fit_NB = classifierNB.fit(train_agr["STATUS"], train_agr["cAGR"])
predicted_NB = classifierNB.predict(test_agr["cAGR"])
scores_NB = cross_validation.cross_val_score(classifierNB, agr["STATUS"], agr["cAGR"], cv = 5)
print("Accuracy NB: %0.3f (+/- %0.2f)" % (scores_NB.mean(), scores_NB.std() * 2))

print("")

clas_fit_kNN = classifier_kNN.fit(train_agr["STATUS"], train_agr["cAGR"])
predicted_kNN = classifier_kNN.predict(test_agr["cAGR"])
scores_kNN = cross_validation.cross_val_score(classifier_kNN, agr["STATUS"], agr["cAGR"], cv = 5)
print("Accuracy of kNN: %0.3f (+/- %0.2f)" % (scores_kNN.mean(), scores_kNN.std() * 2))

print("")

clas_fit_ADC = classifierADC.fit(train_agr["STATUS"], train_agr["cAGR"])
predicted_ADC = classifierADC.predict(test_agr["cAGR"])
scores_ADC = cross_validation.cross_val_score(classifierADC, agr["STATUS"], agr["cAGR"], cv = 5)
print("Accuracy of AdaBoost: %0.3f (+/- %0.2f)" % (scores_ADC.mean(), scores_ADC.std() * 2))

clas_fit_RF = classifierRF.fit(train_agr["STATUS"], train_agr["cAGR"])
predicted_RF = classifierRF.predict(test_agr["cAGR"])
scores_RF = cross_validation.cross_val_score(classifierRF, agr["STATUS"], agr["cAGR"], cv = 5)
print("Accuracy of random Forest: %0.3f (+/- %0.2f)" % (scores_RF.mean(), scores_RF.std() * 2))

Accuracy rbf kernel SVC: 0.531 (+/- 0.00)
Accuracy LinearSVC: 0.511 (+/- 0.02)
Accuracy LinearSVC: 0.492 (+/- 0.04)

Accuracy of bernuli NB: 0.524 (+/- 0.03)
Accuracy NB: 0.525 (+/- 0.02)

Accuracy of kNN: 0.472 (+/- 0.02)

Accuracy of AdaBoost: 0.512 (+/- 0.04)
Accuracy of random Forest: 0.503 (+/- 0.02)


#### VotingClassifier cAGR

http://scikit-learn.org/stable/modules/ensemble.html#votingclassifier

In [14]:
eclf = VotingClassifier(estimators=[
        ("svc", clas_fit_SVC),
        ("linear_svc", clas_fit_LSVC),
        ("nu_svc", clas_fit_NuSVC),
        ("bnb", clas_fit_BNB),
        ("nb", clas_fit_NB),
        ("kNN", clas_fit_kNN), 
        ("rf", clas_fit_RF), 
        ("adc", clas_fit_ADC)], voting='hard')


for clf, label in zip([clas_fit_SVC, clas_fit_LSVC, clas_fit_NuSVC, clas_fit_BNB, clas_fit_NB, clas_fit_kNN, clas_fit_RF, clas_fit_ADC, eclf], ["SVC", "LSVC","NuSVC", "BNB","NB", 'kNN', 'Random Forest', 'ADC', 'Ensemble']):
    scores = cross_validation.cross_val_score(clf, agr["STATUS"], agr["cAGR"], cv=5, scoring='accuracy')
    print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.531 (+/- 0.00) [SVC]
Accuracy: 0.511 (+/- 0.01) [LSVC]
Accuracy: 0.492 (+/- 0.02) [NuSVC]
Accuracy: 0.524 (+/- 0.02) [BNB]
Accuracy: 0.525 (+/- 0.01) [NB]
Accuracy: 0.472 (+/- 0.01) [kNN]
Accuracy: 0.507 (+/- 0.01) [Random Forest]
Accuracy: 0.512 (+/- 0.02) [ADC]
Accuracy: 0.511 (+/- 0.02) [Ensemble]


### cOPN

In [16]:
clas_fit_SVC = classifierSVC.fit(train_opn["STATUS"], train_opn["cOPN"])
predicted_SVC = classifierSVC.predict(test_opn["cOPN"])
scores_SVC = cross_validation.cross_val_score(classifierSVC, opn["STATUS"], opn["cOPN"], cv = 5)
print("Accuracy rbf kernel SVC: %0.3f (+/- %0.2f)" % (scores_SVC.mean(), scores_SVC.std() * 2))
print(sk.metrics.accuracy_score(predicted_SVC, test_opn["cOPN"]))

clas_fit_LSVC = classifierLinearSVC.fit(train_opn["STATUS"], train_opn["cOPN"])
predicted_LSVC = classifierLinearSVC.predict(test_opn["cOPN"])
scores_LSVC = cross_validation.cross_val_score(classifierLinearSVC, opn["STATUS"], opn["cOPN"], cv = 5)
print("Accuracy LinearSVC: %0.3f (+/- %0.2f)" % (scores_LSVC.mean(), scores_LSVC.std() * 2))

clas_fit_NuSVC = classifierNuSVC.fit(train_opn["STATUS"], train_opn["cOPN"])
predicted_NuSVC = classifierNuSVC.predict(test_opn["cOPN"])
scores_NuSVC = cross_validation.cross_val_score(classifierNuSVC, opn["STATUS"], opn["cOPN"], cv = 5)
print("Accuracy NuSVC: %0.3f (+/- %0.2f)" % (scores_NuSVC.mean(), scores_NuSVC.std() * 2))


print("")

clas_fit_BNB = classifierBNB.fit(train_opn["STATUS"], train_opn["cOPN"])
predicted_BNB = classifierBNB.predict(test_opn["cOPN"])
scores_BNB = cross_validation.cross_val_score(classifierBNB, opn["STATUS"], opn["cOPN"], cv = 5)
print("Accuracy of bernuli NB: %0.3f (+/- %0.2f)" % (scores_BNB.mean(), scores_BNB.std() * 2))

clas_fit_NB = classifierNB.fit(train_opn["STATUS"], train_opn["cOPN"])
predicted_NB = classifierNB.predict(test_opn["cOPN"])
scores_NB = cross_validation.cross_val_score(classifierNB, opn["STATUS"], opn["cOPN"], cv = 5)
print("Accuracy NB: %0.3f (+/- %0.2f)" % (scores_NB.mean(), scores_NB.std() * 2))

print("")

clas_fit_kNN = classifier_kNN.fit(train_opn["STATUS"], train_opn["cOPN"])
predicted_kNN = classifier_kNN.predict(test_opn["cOPN"])
scores_kNN = cross_validation.cross_val_score(classifier_kNN, opn["STATUS"], opn["cOPN"], cv = 5)
print("Accuracy of kNN: %0.3f (+/- %0.2f)" % (scores_kNN.mean(), scores_kNN.std() * 2))

print("")

clas_fit_ADC = classifierADC.fit(train_opn["STATUS"], train_opn["cOPN"])
predicted_ADC = classifierADC.predict(test_opn["cOPN"])
scores_ADC = cross_validation.cross_val_score(classifierADC, opn["STATUS"], opn["cOPN"], cv = 5)
print("Accuracy of AdaBoost: %0.3f (+/- %0.2f)" % (scores_ADC.mean(), scores_ADC.std() * 2))

clas_fit_RF = classifierRF.fit(train_opn["STATUS"], train_opn["cOPN"])
predicted_RF = classifierRF.predict(test_opn["cOPN"])
scores_RF = cross_validation.cross_val_score(classifierRF, opn["STATUS"], opn["cOPN"], cv = 5)
print("Accuracy of random Forest: %0.3f (+/- %0.2f)" % (scores_RF.mean(), scores_RF.std() * 2))

Accuracy rbf kernel SVC: 0.743 (+/- 0.00)
0.738434163701
Accuracy LinearSVC: 0.700 (+/- 0.03)
Accuracy NuSVC: 0.502 (+/- 0.10)

Accuracy of bernuli NB: 0.743 (+/- 0.00)
Accuracy NB: 0.744 (+/- 0.00)

Accuracy of kNN: 0.672 (+/- 0.04)

Accuracy of AdaBoost: 0.707 (+/- 0.04)
Accuracy of random Forest: 0.705 (+/- 0.02)


#### VotingClassifier cOPN

http://scikit-learn.org/stable/modules/ensemble.html#votingclassifier

In [17]:
eclf = VotingClassifier(estimators=[
        ("svc", clas_fit_SVC),
        ("linear_svc", clas_fit_LSVC),
        ("nu_svc", clas_fit_NuSVC),
        ("bnb", clas_fit_BNB),
        ("nb", clas_fit_NB),
        ("kNN", clas_fit_kNN), 
        ("rf", clas_fit_RF), 
        ("adc", clas_fit_ADC)], voting='hard')


for clf, label in zip([clas_fit_SVC, clas_fit_LSVC, clas_fit_NuSVC, clas_fit_BNB, clas_fit_NB, clas_fit_kNN, clas_fit_RF, clas_fit_ADC, eclf], ["SVC", "LSVC","NuSVC", "BNB","NB",'kNN', 'Random Forest', 'ADC', 'Ensemble']):
    scores = cross_validation.cross_val_score(clf, opn["STATUS"], opn["cOPN"], cv=5, scoring='accuracy')
    print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.743 (+/- 0.00) [SVC]
Accuracy: 0.700 (+/- 0.02) [LSVC]
Accuracy: 0.502 (+/- 0.05) [NuSVC]
Accuracy: 0.743 (+/- 0.00) [BNB]
Accuracy: 0.744 (+/- 0.00) [NB]
Accuracy: 0.672 (+/- 0.02) [kNN]
Accuracy: 0.705 (+/- 0.01) [Random Forest]
Accuracy: 0.707 (+/- 0.02) [ADC]
Accuracy: 0.737 (+/- 0.01) [Ensemble]


### cCON

In [18]:
clas_fit_SVC = classifierSVC.fit(train_con["STATUS"], train_con["cCON"])
predicted_SVC = classifierSVC.predict(test_con["cCON"])
scores_SVC = cross_validation.cross_val_score(classifierSVC, con["STATUS"], con["cCON"], cv = 5)
print("Accuracy rbf kernel SVC: %0.3f (+/- %0.2f)" % (scores_SVC.mean(), scores_SVC.std() * 2))

clas_fit_LSVC = classifierLinearSVC.fit(train_con["STATUS"], train_con["cCON"])
predicted_LSVC = classifierLinearSVC.predict(test_con["cCON"])
scores_LSVC = cross_validation.cross_val_score(classifierLinearSVC, con["STATUS"], con["cCON"], cv = 5)
print("Accuracy LinearSVC: %0.3f (+/- %0.2f)" % (scores_LSVC.mean(), scores_LSVC.std() * 2))

clas_fit_NuSVC = classifierNuSVC.fit(train_con["STATUS"], train_con["cCON"])
predicted_NuSVC = classifierNuSVC.predict(test_con["cCON"])
scores_NuSVC = cross_validation.cross_val_score(classifierNuSVC, con["STATUS"], con["cCON"], cv = 5)
print("Accuracy NuSVC: %0.3f (+/- %0.2f)" % (scores_NuSVC.mean(), scores_NuSVC.std() * 2))

print("")

clas_fit_BNB = classifierBNB.fit(train_con["STATUS"], train_con["cCON"])
predicted_BNB = classifierBNB.predict(test_con["cCON"])
scores_BNB = cross_validation.cross_val_score(classifierBNB, con["STATUS"], con["cCON"], cv = 5)
print("Accuracy of bernuli NB: %0.3f (+/- %0.2f)" % (scores_BNB.mean(), scores_BNB.std() * 2))

clas_fit_NB = classifierNB.fit(train_con["STATUS"], train_con["cCON"])
predicted_NB = classifierNB.predict(test_con["cCON"])
scores_NB = cross_validation.cross_val_score(classifierNB, con["STATUS"], con["cCON"], cv = 5)
print("Accuracy NB: %0.3f (+/- %0.2f)" % (scores_NB.mean(), scores_NB.std() * 2))

print("")

clas_fit_kNN = classifier_kNN.fit(train_con["STATUS"], train_con["cCON"])
predicted_kNN = classifier_kNN.predict(test_con["cCON"])
scores_kNN = cross_validation.cross_val_score(classifier_kNN, con["STATUS"], con["cCON"], cv = 5)
print("Accuracy of kNN: %0.3f (+/- %0.2f)" % (scores_kNN.mean(), scores_kNN.std() * 2))

print("")

clas_fit_ADC = classifierADC.fit(train_con["STATUS"], train_con["cCON"])
predicted_ADC = classifierADC.predict(test_con["cCON"])
scores_ADC = cross_validation.cross_val_score(classifierADC, con["STATUS"], con["cCON"], cv = 5)
print("Accuracy of AdaBoost: %0.3f (+/- %0.2f)" % (scores_ADC.mean(), scores_ADC.std() * 2))

clas_fit_RF = classifierRF.fit(train_con["STATUS"], train_con["cCON"])
predicted_RF = classifierRF.predict(test_con["cCON"])
scores_RF = cross_validation.cross_val_score(classifierRF, con["STATUS"], con["cCON"], cv = 5)
print("Accuracy of random Forest: %0.3f (+/- %0.2f)" % (scores_RF.mean(), scores_RF.std() * 2))

Accuracy rbf kernel SVC: 0.541 (+/- 0.00)
Accuracy LinearSVC: 0.526 (+/- 0.05)
Accuracy LinearSVC: 0.500 (+/- 0.05)

Accuracy of bernuli NB: 0.539 (+/- 0.04)
Accuracy NB: 0.543 (+/- 0.04)

Accuracy of kNN: 0.512 (+/- 0.02)

Accuracy of AdaBoost: 0.533 (+/- 0.03)
Accuracy of random Forest: 0.527 (+/- 0.03)


#### VotingClassifier cCON

http://scikit-learn.org/stable/modules/ensemble.html#votingclassifier

In [19]:
eclf = VotingClassifier(estimators=[
        ("svc", clas_fit_SVC),
        ("linear_svc", clas_fit_LSVC), 
        ("nu_svc", clas_fit_NuSVC),
        ("bnb", clas_fit_BNB),
        ("nb", clas_fit_NB),
        ("kNN", clas_fit_kNN), 
        ("rf", clas_fit_RF), 
        ("adc", clas_fit_ADC)], voting='hard')


for clf, label in zip([clas_fit_SVC, clas_fit_LSVC, clas_fit_NuSVC, clas_fit_BNB, clas_fit_NB, clas_fit_kNN, clas_fit_RF, clas_fit_ADC, eclf], ["SVC", "LSVC","NuSVC","BNB","NB",'kNN', 'Random Forest', 'ADC', 'Ensemble']):
    scores = cross_validation.cross_val_score(clf, con["STATUS"], con["cCON"], cv=5, scoring='accuracy')
    print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.541 (+/- 0.00) [SVC]
Accuracy: 0.526 (+/- 0.02) [LSVC]
Accuracy: 0.500 (+/- 0.02) [NuSVC]
Accuracy: 0.539 (+/- 0.02) [BNB]
Accuracy: 0.543 (+/- 0.02) [NB]
Accuracy: 0.512 (+/- 0.01) [kNN]
Accuracy: 0.534 (+/- 0.01) [Random Forest]
Accuracy: 0.533 (+/- 0.02) [ADC]
Accuracy: 0.543 (+/- 0.01) [Ensemble]


### cEXT

In [20]:
clas_fit_SVC = classifierSVC.fit(train_ext["STATUS"], train_ext["cEXT"])
predicted_SVC = classifierSVC.predict(test_ext["cEXT"])
scores_SVC = cross_validation.cross_val_score(classifierSVC, ext["STATUS"], ext["cEXT"], cv = 5)
print("Accuracy rbf kernel SVC: %0.3f (+/- %0.2f)" % (scores_SVC.mean(), scores_SVC.std() * 2))

clas_fit_LSVC = classifierLinearSVC.fit(train_ext["STATUS"], train_ext["cEXT"])
predicted_LSVC = classifierLinearSVC.predict(test_ext["cEXT"])
scores_LSVC = cross_validation.cross_val_score(classifierLinearSVC, ext["STATUS"], ext["cEXT"], cv = 5)
print("Accuracy LinearSVC: %0.3f (+/- %0.2f)" % (scores_LSVC.mean(), scores_LSVC.std() * 2))

clas_fit_NuSVC = classifierNuSVC.fit(train_ext["STATUS"], train_ext["cEXT"])
predicted_NuVC = classifierNuSVC.predict(test_ext["cEXT"])
scores_NuSVC = cross_validation.cross_val_score(classifierNuSVC, ext["STATUS"], ext["cEXT"], cv = 5)
print("Accuracy NuSVC: %0.3f (+/- %0.2f)" % (scores_NuSVC.mean(), scores_NuSVC.std() * 2))

print("")

clas_fit_BNB = classifierBNB.fit(train_ext["STATUS"], train_ext["cEXT"])
predicted_BNB = classifierBNB.predict(test_ext["cEXT"])
scores_BNB = cross_validation.cross_val_score(classifierBNB, ext["STATUS"], ext["cEXT"], cv = 5)
print("Accuracy of bernuli NB: %0.3f (+/- %0.2f)" % (scores_BNB.mean(), scores_BNB.std() * 2))

clas_fit_NB = classifierNB.fit(train_ext["STATUS"], train_ext["cEXT"])
predicted_NB = classifierNB.predict(test_ext["cEXT"])
scores_NB = cross_validation.cross_val_score(classifierNB, ext["STATUS"], ext["cEXT"], cv = 5)
print("Accuracy NB: %0.3f (+/- %0.2f)" % (scores_NB.mean(), scores_NB.std() * 2))

print("")

clas_fit_kNN = classifier_kNN.fit(train_ext["STATUS"], train_ext["cEXT"])
predicted_kNN = classifier_kNN.predict(test_ext["cEXT"])
scores_kNN = cross_validation.cross_val_score(classifier_kNN, ext["STATUS"], ext["cEXT"], cv = 5)
print("Accuracy of kNN: %0.3f (+/- %0.2f)" % (scores_kNN.mean(), scores_kNN.std() * 2))

print("")

clas_fit_ADC = classifierADC.fit(train_ext["STATUS"], train_ext["cEXT"])
predicted_ADC = classifierADC.predict(test_ext["cEXT"])
scores_ADC = cross_validation.cross_val_score(classifierADC, ext["STATUS"], ext["cEXT"], cv = 5)
print("Accuracy of AdaBoost: %0.3f (+/- %0.2f)" % (scores_ADC.mean(), scores_ADC.std() * 2))

clas_fit_RF = classifierRF.fit(train_ext["STATUS"], train_ext["cEXT"])
predicted_RF = classifierRF.predict(test_ext["cEXT"])
scores_RF = cross_validation.cross_val_score(classifierRF, ext["STATUS"], ext["cEXT"], cv = 5)
print("Accuracy of random Forest: %0.3f (+/- %0.2f)" % (scores_RF.mean(), scores_RF.std() * 2))

Accuracy rbf kernel SVC: 0.575 (+/- 0.00)
Accuracy LinearSVC: 0.538 (+/- 0.01)
Accuracy NuSVC: 0.522 (+/- 0.10)

Accuracy of bernuli NB: 0.571 (+/- 0.03)
Accuracy NB: 0.568 (+/- 0.01)

Accuracy of kNN: 0.523 (+/- 0.02)

Accuracy of AdaBoost: 0.557 (+/- 0.02)
Accuracy of random Forest: 0.551 (+/- 0.01)


#### VotingClassifier cEXT

http://scikit-learn.org/stable/modules/ensemble.html#votingclassifier

In [21]:
eclf = VotingClassifier(estimators=[
        ("svc", clas_fit_SVC),
        ("linear_svc", clas_fit_LSVC), 
        ("nu_svc", clas_fit_NuSVC),
        ("bnb", clas_fit_BNB),
        ("nb", clas_fit_NB),
        ("kNN", clas_fit_kNN), 
        ("rf", clas_fit_RF), 
        ("adc", clas_fit_ADC)], voting='hard')


for clf, label in zip([clas_fit_SVC, clas_fit_LSVC, clas_fit_NuSVC, clas_fit_BNB, clas_fit_NB, clas_fit_kNN, clas_fit_RF, clas_fit_ADC, eclf], ["SVC", "LSVC", "NuSVC", "BNB", "NB", 'kNN', 'Random Forest', 'ADC', 'Ensemble']):
    scores = cross_validation.cross_val_score(clf, ext["STATUS"], ext["cEXT"], cv=5, scoring='accuracy')
    print("Accuracy: %0.3f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.575 (+/- 0.00) [SVC]
Accuracy: 0.538 (+/- 0.01) [LSVC]
Accuracy: 0.522 (+/- 0.05) [NuSVC]
Accuracy: 0.571 (+/- 0.01) [BNB]
Accuracy: 0.568 (+/- 0.00) [NB]
Accuracy: 0.523 (+/- 0.01) [kNN]
Accuracy: 0.546 (+/- 0.01) [Random Forest]
Accuracy: 0.557 (+/- 0.01) [ADC]
Accuracy: 0.571 (+/- 0.01) [Ensemble]
